# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment

import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name="./Reacher_Windows_x86_64/Reacher.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [71]:
import math
from torch.autograd import Variable


pi = Variable(torch.FloatTensor([math.pi]), requires_grad=True).cuda()

def normal(x, mu, sigma_sq):
    a = (-1*(Variable(x, requires_grad=True)-mu).pow(2)/(2*sigma_sq)).exp()
    b = 1/(2*sigma_sq*pi.expand_as(sigma_sq)).sqrt()
    return a*b

class Policy(nn.Module):

    def __init__(self, state_size, action_size, fc1_size=128, fc2_size=256):
        super(Policy, self).__init__()
        
        self.fc1 = nn.Linear(33, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        
        self.fc3_mu = nn.Linear(fc2_size, action_size)
        self.fc3_sigma_squared = nn.Linear(fc2_size, action_size)
        
        self.hardtanh = torch.nn.Hardtanh()
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        mu = self.hardtanh(self.fc3_mu(x))
        sigma_squared = F.softplus(self.fc3_sigma_squared(x).abs())
        
        return mu, sigma_squared


In [72]:
policy=Policy(state_size=33, action_size=action_size).to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-4)

### 5. REINFORCE

In [109]:
def probs_to_actions(mu, sigma_sq):
    eps = torch.randn(mu.size())
    action = (mu + sigma_sq.sqrt()*Variable(eps, requires_grad=True).cuda())
    prob = normal(action, mu, sigma_sq)
    
    return action, prob

def collect_trajectories(env, brain_name, policy, t_max=200):
    """
        Unroll a trajectory and return 
        list of action probabilities, states, actions and rewards

        Parameters
        ----------
        env: unityagents.UnityEnvironment
        Environment to play on.
        brain_name: String
        Name of brain used for UnityEnvironment
        policy: torch.nn.Module
        A neural network mapping states to action probabilities 
        The action to take at a given state
        t_max: int
        Maximum number of episodes in trajectory
    """
    
    env_info = env.reset(train_mode=True)[brain_name]
    # num_agents = len(env.ps)

    state_list  = []
    reward_list = []
    prob_list   = []
    action_list = []

    states = env_info.vector_observations
    
    for _ in range(t_max):
        
        mu, sigma = policy(torch.from_numpy(states).float().to(device))   
        actions, probabilities = probs_to_actions(mu, sigma)
        
        actions       = actions.detach().cpu().numpy()
        probabilities = probabilities.detach().cpu().numpy()

        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        
        prob_list.append(probabilities)
        state_list.append(states)
        action_list.append(actions)
        reward_list.append(env_info.rewards)

        if np.any(env_info.local_done):
            break

        states = next_states

    return prob_list, state_list, action_list, reward_list

In [110]:
trajectory = collect_trajectories(env, brain_name, policy, t_max=200)

In [113]:
def REINFORCE(policy, old_probs, states, actions, rewards, discount = 0.995, beta=0.01):

    discount = discount**np.arange(len(rewards))
    rewards = np.asarray(rewards)*discount[:,np.newaxis]
    
    # convert rewards to future rewards
    rewards_future = rewards[::-1].cumsum(axis=0)[::-1]
    
    mean = np.mean(rewards_future, axis=1)
    std = np.std(rewards_future, axis=1) + 1.0e-10

    rewards_normalized = (rewards_future - mean[:,np.newaxis])/std[:,np.newaxis]
    
    # convert everything into pytorch tensors and move to gpu if available
    actions = torch.tensor(actions, dtype=torch.int8, device=device)
    old_probs = torch.tensor(old_probs, dtype=torch.float, device=device)
    rewards = torch.tensor(rewards_normalized, dtype=torch.float, device=device)
    states  = torch.tensor(states, dtype=torch.float, device=device) 
    
    # convert states to policy (or probability)
    mu, sigma = policy(states)
    
    print(mu)
    print(sigma)
    
    new_actions, new_probs = probs_to_actions(mu, sigma)
    new_probs = torch.tensor(new_probs, dtype=torch.float, device=device)
    
    return torch.mean(rewards * torch.log(old_probs))

In [114]:
import progressbar as pb

# HYPERPARAMETERS
episode = 500
discount_rate = .99
beta = .01
tmax = 320

# widget bar to display progress
widget = ['training loop: ', pb.Percentage(), ' ', pb.Bar(), ' ', pb.ETA()]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()

# keep track of progress
mean_rewards = []

for e in range(episode):

    old_probs, states, actions, rewards = collect_trajectories(env, brain_name, policy, t_max=tmax)
        
    total_rewards = np.sum(rewards, axis=0)

    policy.train()
    L = -REINFORCE(policy, old_probs, states, actions, rewards, beta=beta)
    
    optimizer.zero_grad()
    L.backward()
    optimizer.step()
        
    beta *= .995
    
    mean_rewards.append(np.mean(total_rewards))
    
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    timer.update(e+1)
    
timer.finish()
    

tensor([[[ 0.0261, -0.1765, -0.2755,  0.1381]],

        [[-0.2593, -0.1426, -0.3026,  0.1372]],

        [[-0.0826, -0.2560, -0.4096,  0.1332]],

        ...,

        [[ 0.0919,  0.0706, -0.5485, -0.0656]],

        [[-0.0102, -0.0904, -0.4387, -0.0108]],

        [[ 0.0041,  0.0966, -0.4637, -0.0128]]], device='cuda:0',
       grad_fn=<HardtanhBackward0>)
tensor([[[0.8003, 0.9347, 0.8024, 0.7912]],

        [[0.7452, 1.0107, 0.8252, 0.7480]],

        [[0.7576, 0.9529, 0.8796, 0.7695]],

        ...,

        [[0.7400, 1.0493, 0.6948, 0.7768]],

        [[0.7244, 0.9385, 0.7177, 0.8443]],

        [[0.7752, 1.0647, 0.7633, 0.7587]]], device='cuda:0',
       grad_fn=<SoftplusBackward>)


D:\Anaconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn